# Importer les bibliothèques nécessaires

# Step 0: We import the required libraries 

In [543]:
import pandas as pd
import ast 
import numpy as np
import csv

# Step 1: We import the data exported from Doctolib  

In [544]:
data = pd.read_csv('data/extracted_data_final_0_1000.csv')

# Step 2: We reorganize the data exported from Doctolib 

In [545]:
# Function to reorganize the profile cards 
def reorder_cards(profile_cards):
    try:
        cards_dict = ast.literal_eval(profile_cards)
        
        reordered_cards = {
            'card_0': np.nan,
            'card_1': np.nan,
            'card_2': np.nan,
            'card_3': np.nan,
            'card_4': np.nan,
            'card_5': np.nan
        }

        def split_content(value):
            value = value.replace('\\n', '\n').replace('\\', '\n')
            return value.split('\n')

        for key, value in cards_dict.items():
            if value.startswith("Carte et informations d'accès"):
                reordered_cards['card_1'] = split_content(value)
            elif value.startswith("Présentation"):
                reordered_cards['card_2'] = split_content(value)
            elif value.startswith("Horaires et coordonnées"):
                reordered_cards['card_3'] = split_content(value)
            elif value.startswith("Tarifs et remboursement") or value.startswith("Lieu"):
                reordered_cards['card_0'] = split_content(value)
            elif value.startswith("Tarifs"):
                reordered_cards['card_4'] = split_content(value)
            elif value.startswith("Informations légales"):
                reordered_cards['card_5'] = split_content(value)
            else :
                if isinstance(value,list):
                    reordered_cards['card_0'] = split_content(value)
        
        return reordered_cards
    except:
        return {
            'card_0': np.nan,
            'card_1': np.nan,
            'card_2': np.nan,
            'card_3': np.nan,
            'card_4': np.nan,
            'card_5': np.nan
        }

reordered_cards = data['Profile Cards'].apply(reorder_cards)

reordered_cards_df = pd.json_normalize(reordered_cards)

data_combined = pd.concat([data, reordered_cards_df], axis=1)

data_combined.drop(columns = 'Profile Cards', inplace=True)


In [546]:
#To provide a clearer localisation
def extract_lat_lng(map_location):
    try:
        location_dict = ast.literal_eval(map_location)
        return (location_dict['lat'], location_dict['lng'])
    except:
        return (None, None)

data_combined['Latitude'], data_combined['Longitude'] = zip(*data_combined['Map Location'].apply(extract_lat_lng))

data_combined.drop(columns = 'Map Location', inplace=True)


In [547]:
# Function to treat the column 'Skills'
def process_skills(skills):
    if skills == 'Skills missing':
        return np.nan
    else:
        return skills.split('\n')

data_combined['Skills Processed'] = data_combined['Skills'].apply(process_skills)

data_combined.drop(columns = 'Skills', inplace=True)

In [548]:
#To check which specialty has more doctors
data_combined['Specialty'].value_counts()

Specialty
Chirurgien-dentiste                                                 152
Médecin généraliste                                                 104
Ophtalmologue                                                        75
Cabinet médical                                                      56
Spécialiste en chirurgie plastique reconstructrice et esthétique     40
                                                                   ... 
Chirurgien oral                                                       1
Acupuncteur                                                           1
Addictologue                                                          1
Chirurgien cancérologue                                               1
Diététicien                                                           1
Name: count, Length: 78, dtype: int64

In [549]:
data_combined['RPPS'] = data_combined['card_5'].apply(lambda x: x[2] if isinstance(x, list) and len(x) > 2 else np.nan)
data_combined.drop(columns = 'card_5', inplace = True)

In [550]:
data_combined['Nb Skills'] = data_combined['Skills Processed'].apply(lambda x: len(x)-1 if isinstance(x, list) else 0)

In [551]:
def paiement(card0):
    if isinstance(card0, list):
        if 'Moyens de paiement' in card0:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['moyens_paiement'] = data_combined['card_0'].apply(paiement)


def vitale(card0):
    if isinstance(card0, list):
        if 'Carte Vitale acceptée' in card0:
            return 1
        elif 'Carte Vitale non acceptée' in card0:
            return 0
        else:
            return np.nan
    else:
        return np.nan
data_combined['carte_vitale'] = data_combined['card_0'].apply(paiement)


def secteur(card0):
    if isinstance(card0, list):
        a=0
        for x in card0:
            if 'Conventionné secteur 2' in x:
                a=2
            elif 'Conventionné secteur 1' in x:
                a=1
            elif 'Conventionné secteur 3' in x:
                a=3
            elif 'Conventionné' in x:
                a=4
        return a
    else:
        return 0
data_combined['secteur'] = data_combined['card_0'].apply(secteur)

In [552]:
data_combined['Présentation']=''
data_combined['Langues parlées']=''
data_combined['Diplômes nationaux et universitaires']=''
data_combined['Autres formations']=''
data_combined['Expériences']=''
data_combined['Travaux et publications']=''
data_combined['Prix et distinctions']=''
data_combined['site']=0

liste_mots_clefs = ['Présentation','Langues parlées','Diplômes nationaux et universitaires','Formations','Autres formations','Expériences', 'Site web', 'Travaux et publications', 'Prix et distinctions']

for i in range(data_combined.shape[0]):
    if isinstance(data_combined['card_2'].iloc[i],list) : 
        liste_card2 = data_combined['card_2'].iloc[i][1:]
        sous_liste = ''
        label = liste_mots_clefs[0]
        for x in liste_card2 :
            if x in liste_mots_clefs :
                data_combined.loc[i, label] = sous_liste
                label = x
                sous_liste = ''
            elif x=='Voir le site':
                data_combined.loc[i, 'site'] = 1
            elif x=='▾ Voir plus':
                None
            else :
                sous_liste = sous_liste+(x)

In [553]:
data_combined['Diplômes nationaux et universitaires b']=data_combined['Diplômes nationaux et universitaires'].apply(lambda x: 1 if len(x)>0 else 0)

In [554]:
data_combined['Nb caractères présentation'] = data_combined['Présentation'].apply(lambda x: len(x))

In [555]:
data_combined['Autres formations b']=data_combined['Autres formations'].apply(lambda x: 1 if len(x)>0 else 0)

In [556]:
data_combined['Travaux et publications b']=data_combined['Travaux et publications'].apply(lambda x: 1 if len(x)>0 else 0)

In [557]:
data_combined['Nb langues']=0

for i in range(data_combined.shape[0]):
    if isinstance(data_combined['Langues parlées'].iloc[i],str) : 
        liste_langue = (data_combined['Langues parlées'].iloc[i]).split()
        count = 0
        for x in liste_langue :
            
            if  x != 'et'  :
                count +=1
        data_combined.loc[i,'Nb langues']=count

In [558]:
data_combined['Expériences b']=data_combined['Expériences'].apply(lambda x: 1 if len(x)>0 else 0)

In [559]:
data_combined['Formations b']=data_combined['Formations'].apply(lambda x: 1 if isinstance(x,str) else 0)

In [560]:
liste1 = ["Carte et informations d'accès",'Moyens de transport','Parking public','Informations pratiques',"J'autorise le traitement d'informations (dont mon adresse IP) et leur transfert hors UE par Google Maps (USA) afin d’afficher la carte.En savoir plus", 'sur la collecte et le traitement des données par Google','AFFICHER LA CARTE']
def carac1(card1):
    nb=0
    for phrase in card1:
        if  phrase not in liste1:
            nb += len(phrase)
    return nb
data_combined['Carac_card_1'] = data_combined['card_1'].apply(lambda x: carac1(x)if isinstance(x, list) else np.nan)

In [561]:
def transport(card1):
    if isinstance(card1, list):
        if 'Moyens de transport' in card1:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Transport'] = data_combined['card_1'].apply(transport)

In [562]:
def parking(card1):
    if isinstance(card1, list):
        if 'Parking public' in card1:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Parking'] = data_combined['card_1'].apply(parking)

In [563]:
def infos(card1):
    if isinstance(card1, list):
        if 'Informations pratiques' in card1:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Infos'] = data_combined['card_1'].apply(infos)

In [564]:
def coordonnées(card3):
    if isinstance(card3, list):
        if 'Coordonnées' in card3:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Coordonnées'] = data_combined['card_3'].apply(coordonnées) 

In [565]:
def sansRDV(card3):
    if isinstance(card3, list):
        if 'Consultations sans rendez-vous' in card3:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Sans RDV'] = data_combined['card_3'].apply(sansRDV)

In [566]:
data_combined.drop(columns=['URL','card_1','card_2'],inplace=True)
data_combined.head()

,Doctor Name,Specialty,card_0,card_3,card_4,Latitude,Longitude,Skills Processed,RPPS,Nb Skills,...,Travaux et publications b,Nb langues,Expériences b,Formations b,Carac_card_1,Transport,Parking,Infos,Coordonnées,Sans RDV
0,Dr Marian AGACHI,Neurochirurgien,NaN,"[Horaires et coordonnées, Horaires d'ouverture...",NaN,48.878328,2.431034,"[Expertises et actes, Neurochirurgie du rachis]",10001649291,1,...,0,3,0,1,154.0,1,0,1,1,0
1,Dr Anne Vaillant Moga,Médecin généraliste,"[Lieu 1, Lieu 2, Dr Anne Moga (Paris), 76 aven...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation préalable de médecine es...",48.869218,2.285468,"[Expertises et actes, Acné, Allergie, Apnée du...",10000378991,10,...,1,2,1,0,180.0,1,0,1,0,0
2,Dr Gabriel Ohana,Chirurgien-dentiste,"[Tarifs et remboursement, , Carte Vitale accep...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation dentaire, 23 €, Traiteme...",48.894148,2.250698,"[Expertises et actes, Chirurgie buccale, Proth...",10003639373,3,...,0,1,0,0,240.0,1,0,1,1,0
3,Dr Thomas BAMBERGER,Dermatologue et vénérologue,"[Tarifs et remboursement, , , Conventionné sec...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation, 31,50 € à 56,50 €, Cont...",48.815433,2.343017,NaN,10003719571,0,...,0,1,1,0,155.0,1,0,1,1,0
4,Dr Layla Chatila,Chirurgien-dentiste,"[Tarifs et remboursement, , , , Conventionné, ...","[Horaires et coordonnées, Coordonnées, 01 45 2...","[Tarifs, Consultation dentaire, 23 €, Ces hono...",48.858804,2.274392,NaN,10005133003,0,...,0,1,0,0,202.0,1,1,1,1,0


In [567]:
def nb_terms(card):
    if type(card)==list :
        return (len(card))
    else :
        return(0)

def nb_carac(card):
    if type(card)==list :
        s = 0
        for item in card : 
            s+=len(card)
        return(s)
    else :
        return(0)

In [568]:
data_combined['nb terms card_0']=data_combined['card_0'].apply(nb_terms)
data_combined['nb terms card_3']=data_combined['card_3'].apply(nb_terms)
data_combined['nb terms card_4']=data_combined['card_4'].apply(nb_terms)
data_combined['nb carac card_0']=data_combined['card_0'].apply(nb_carac)
data_combined['nb carac card_3']=data_combined['card_3'].apply(nb_carac)
data_combined['nb carac card_4']=data_combined['card_4'].apply(nb_carac)

# Step 3: We separate the doctor whom we know the RPPS from the others  

In [569]:
indices_nan = data_combined[data_combined['RPPS'].isna()].index.tolist()
print(indices_nan, len(indices_nan))

[8, 23, 27, 73, 74, 94, 112, 116, 117, 118, 121, 122, 123, 124, 127, 130, 131, 132, 135, 136, 138, 139, 140, 141, 142, 143, 145, 148, 150, 151, 152, 153, 156, 157, 158, 160, 161, 163, 166, 167, 169, 170, 172, 173, 175, 176, 177, 178, 179, 180, 182, 183, 184, 186, 187, 188, 190, 191, 192, 194, 196, 197, 198, 201, 202, 204, 209, 213, 214, 215, 216, 219, 220, 221, 222, 223, 225, 226, 227, 229, 230, 231, 233, 234, 235, 236, 238, 239, 240, 241, 245, 246, 247, 249, 250, 252, 254, 256, 259, 260, 262, 263, 264, 265, 267, 268, 270, 271, 272, 276, 277, 278, 279, 280, 281, 282, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 312, 315, 316, 317, 318, 319, 320, 321, 323, 324, 325, 326, 327, 328, 329, 331, 335, 337, 339, 344, 346, 347, 348, 349, 350, 351, 352, 353, 354, 356, 358, 359, 360, 365, 367, 368, 371, 372, 373, 374, 376, 378, 379, 380, 381, 382, 383, 384, 387, 388, 390, 392, 393, 395, 396, 397, 398, 399, 400, 4

In [570]:
data['Profile Cards'].iloc[999]

'{\'card_0\': \'Tarifs et remboursement\\n\\n\\n\\nConventionné\\nCarte Vitale acceptée\\nTiers payant : Sécurité sociale\\nVoir les tarifs\\nMoyens de paiement\\nChèques, espèces et carte bancaire\\nExpertises et actes\\nBilan podologique\\nIonophorèse\\nK-Tape\\nKinesio Taping\\nOrthonyxie\\nOrthoplastie\\nOrthèse plantaire\\nPodologie\\nPédicurie\\nSemelles orthopédiques\\nVerrue\', \'card_1\': "Carte et informations d\'accès\\n9 Avenue Ampère, 77420 Champs-sur-Marne\\nMoyens de transport\\nBus - Ampere (lignes 213 et 312)\\nBus - Noisy-Champs Rer - Descartes (ligne 212)\\nInformations pratiques\\n1er étage avec ascenseur\\nEntrée accessible\\nJ\'autorise le traitement d\'informations (dont mon adresse IP) et leur transfert hors UE par Google Maps (USA) afin d’afficher la carte.En savoir plus\\nsur la collecte et le traitement des données par Google\\nAFFICHER LA CARTE", \'card_2\': "Présentation\\nDepuis janvier 2010, je vous reçois dans mon cabinet à Champs-sur-Marne à deux pas de

In [571]:
data_combined_sans_nan = data_combined[data_combined['RPPS'].notna()]
data_combined_sans_nan

,Doctor Name,Specialty,card_0,card_3,card_4,Latitude,Longitude,Skills Processed,RPPS,Nb Skills,...,Parking,Infos,Coordonnées,Sans RDV,nb terms card_0,nb terms card_3,nb terms card_4,nb carac card_0,nb carac card_3,nb carac card_4
0,Dr Marian AGACHI,Neurochirurgien,NaN,"[Horaires et coordonnées, Horaires d'ouverture...",NaN,48.878328,2.431034,"[Expertises et actes, Neurochirurgie du rachis]",10001649291,1,...,0,1,1,0,0,8,0,0,64,0
1,Dr Anne Vaillant Moga,Médecin généraliste,"[Lieu 1, Lieu 2, Dr Anne Moga (Paris), 76 aven...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation préalable de médecine es...",48.869218,2.285468,"[Expertises et actes, Acné, Allergie, Apnée du...",10000378991,10,...,0,1,0,0,25,6,32,625,36,1024
2,Dr Gabriel Ohana,Chirurgien-dentiste,"[Tarifs et remboursement, , Carte Vitale accep...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation dentaire, 23 €, Traiteme...",48.894148,2.250698,"[Expertises et actes, Chirurgie buccale, Proth...",10003639373,3,...,0,1,1,0,12,8,10,144,64,100
3,Dr Thomas BAMBERGER,Dermatologue et vénérologue,"[Tarifs et remboursement, , , Conventionné sec...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation, 31,50 € à 56,50 €, Cont...",48.815433,2.343017,NaN,10003719571,0,...,0,1,1,0,8,8,10,64,64,100
4,Dr Layla Chatila,Chirurgien-dentiste,"[Tarifs et remboursement, , , , Conventionné, ...","[Horaires et coordonnées, Coordonnées, 01 45 2...","[Tarifs, Consultation dentaire, 23 €, Ces hono...",48.858804,2.274392,NaN,10005133003,0,...,1,1,1,0,9,3,4,81,9,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,Dr Danielle Taconet,Ophtalmologue,"[Tarifs et remboursement, , , Conventionné sec...","[Horaires et coordonnées, Contact d'urgence, E...",NaN,49.060704,2.184674,"[Expertises et actes, Champ visuel, Laser opht...",10003749230,4,...,0,1,1,0,13,5,0,169,25,0
994,Dr Stella Danan-Hadida,Gynécologue médicale et obstétrique,"[Tarifs et remboursement, , , , Conventionné s...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation vidéo, 58 €, Consultatio...",48.871909,2.346805,"[Expertises et actes, Gynécologie obstétrique,...",10000292291,7,...,1,1,1,0,18,8,28,324,64,784
995,Dr Philippe Lazare SELLAM,Spécialiste en chirurgie plastique reconstruct...,"[Lieu 1, Lieu 2, Cabinet du Docteur Philippe L...","[Horaires et coordonnées, Contact d'urgence, E...","[Tarifs, Consultation, 23 € à 120 €, Exérèse d...",48.868676,2.325944,NaN,10000558964,0,...,1,1,1,0,11,5,8,121,25,64
996,Dr Michèle Maizil,Chirurgien-dentiste,NaN,"[Horaires et coordonnées, Horaires d'ouverture...",NaN,48.834970,2.246637,NaN,10000202985,0,...,1,1,1,0,0,8,0,0,64,0


# Step 4: We open the data from “Annuaire Santé”  

In [572]:
annuaire = pd.read_csv('data/PS_LibreAcces_Personne_activite_small.csv')
annuaire.shape

/var/folders/rf/dqdwsk6s06ngm9b4j17fkb2r0000gn/T/ipykernel_29677/2401431438.py:1: DtypeWarning: Columns (10,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  annuaire = pd.read_csv('data/PS_LibreAcces_Personne_activite_small.csv')


(83743, 27)

In [573]:
#To verify our database
annuaire.isnull().sum()

identifiantpp                           0
identificationnationalepp               0
codecivilité                            0
nomdexercice                            1
prénomdexercice                         1
codecatégorieprofessionnelle            0
codesavoirfaire                         0
libellésavoirfaire                      0
codemodeexercice                        0
numérosiretsite                     58015
numérofinesssite                    57014
identifianttechniquedelastructur        0
raisonsocialesite                       0
numérovoiecoordstructure            10187
indicerépétitionvoiecoordstructu    78376
codetypedevoiecoordstructure        57627
libellétypedevoiecoordstructure     57627
libellévoiecoordstructure            1195
bureaucedexcoordstructure               0
codepostalcoordstructure                0
codecommunecoordstructure               1
libellécommunecoordstructure            1
libellépayscoordstructure           26729
codesecteurdactivité              

In [574]:
annuaire['identifiantpp']=annuaire['identifiantpp'].astype(str)

# Step 5: We focus on the general practitioner 

In [575]:
medecin_gen = data_combined_sans_nan[data_combined_sans_nan['Specialty']=='Médecin généraliste']

In [577]:
liste_med = annuaire.merge(medecin_gen,how='outer',left_on='identifiantpp',right_on='RPPS')

In [578]:
def departement(codepostal):
    code = int(codepostal)/1000
    if int(code) == 97 :
        return(str(int(codepostal/100)))
    if int(code) < 10 :
        return('0'+str(int(code)))
    return (str(int(code)))

In [579]:
liste_med.loc[:, 'code dépar'] = liste_med['codepostalcoordstructure'].apply(departement)

# Step 6: A first measure of competition 

In [581]:
concu = pd.read_excel('data/TCRD_068.xlsx')
concu.rename(columns={'Professionnels de santé au 1ᵉʳ janvier 2023 : comparaisons départementales' : 'Code Département','Unnamed: 1' : 'Nom Département','Unnamed: 2':'Nombre de médecins','Unnamed: 3':'Nombre de médecins pour 100 000 habitants','Unnamed: 4':'généralistes','Unnamed: 5':'spécialistes','Unnamed: 6':'dentistes','Unnamed: 7':'Pharm.'}, inplace=True)
concu.drop(labels = [0,1,2,3,108],axis=0, inplace=True)
concu

,Code Département,Nom Département,Nombre de médecins,Nombre de médecins pour 100 000 habitants,généralistes,spécialistes,dentistes,Pharm.
4,01,Ain,1162,174,99,75,53,78
5,02,Aisne,1107,211,99,111,45,91
6,03,Allier,917,275,134,141,51,111
7,04,Alpes-de-Haute-Provence,483,291,165,125,54,103
8,05,Hautes-Alpes,705,503,291,213,72,135
...,...,...,...,...,...,...,...,...
103,972,Martinique,1171,332,152,180,68,99
104,973,Guyane,717,242,123,120,30,46
105,974,La Réunion,3170,364,176,189,67,94
106,976,Mayotte,265,89,49,39,9,31


In [582]:
liste_med = liste_med.merge(concu[['Code Département','généralistes']], left_on='code dépar', right_on='Code Département',how='inner')

In [583]:
data_final = liste_med[['nomdexercice', 'prénomdexercice','RPPS','Specialty','card_0', 'card_3', 'card_4','Skills Processed','Nb Skills', 'moyens_paiement',
       'carte_vitale', 'secteur', 'Présentation', 'Langues parlées',
       'Diplômes nationaux et universitaires', 'Autres formations',
       'Expériences', 'Travaux et publications',
       'Prix et distinctions', 'site', 'Formations',
       'Diplômes nationaux et universitaires b', 'Nb caractères présentation',
       'Autres formations b', 'Nb langues', 'Expériences b', 'Carac_card_1',
       'Transport', 'Parking', 'Infos', 'Coordonnées', 'Sans RDV','Code Département',
       'généralistes']]
data_final.head()

,nomdexercice,prénomdexercice,RPPS,Specialty,card_0,card_3,card_4,Skills Processed,Nb Skills,moyens_paiement,...,Nb langues,Expériences b,Carac_card_1,Transport,Parking,Infos,Coordonnées,Sans RDV,Code Département,généralistes
0,VUILLET,MICHEL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,971,150
1,VUILLET,MICHEL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,971,150
2,WEILL,EDGARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,971,150
3,BORDESSOULES,PIERRE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,971,150
4,DORVILLE,ALBERT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,971,150


# Step 7: A second measure of competition 

In [584]:
liste_med_ban=liste_med[liste_med['numérovoiecoordstructure'].notnull()]

In [585]:
def nom_voie(nom_voie):
    if isinstance(nom_voie,str):
        if nom_voie[0] in ['1','2','3','4','5','6','7','8','9']:
            nom_voie = nom_voie[1:]
        if 'Imp ' in nom_voie:
            return nom_voie.replace('Imp ','Impasse ').lower()
        else:
            return nom_voie.lower()
    else:
        return nom_voie.lower()

In [586]:
liste_med_ban=liste_med_ban[liste_med_ban['libellépayscoordstructure']!='Belgique']

In [587]:
def fct_voie(type_voie,nom_voie):
    result = ''
    if isinstance(type_voie,str) and isinstance(nom_voie,str):
        result = (type_voie+' '+nom_voie).lower()
    elif isinstance(nom_voie,str):
        result = nom_voie.lower()
    else:
        result = nom_voie
    if isinstance(result,str) and result[0]=='-':
        if result[1]==' ':
            result= result[2:]
        else :
            result = result[1:]
    if isinstance(result,str) and result[0]=='/':
        result = result[1:]
    return result

In [588]:
liste_med_ban['Voie'] = liste_med_ban.apply(lambda x: fct_voie(x['libellétypedevoiecoordstructure'],x['libellévoiecoordstructure']),axis=1)

In [589]:
#Gestion des erreurs de merge
def codecom3(codecom1):
    if str(codecom1).startswith('75'):
        return('75056')
    elif str(codecom1).startswith('693'):
        return('69123')
    elif str(codecom1).startswith('130') or str(codecom1).startswith('132'):
        return('13055')
    elif str(codecom1).startswith('74011') or str(codecom1).startswith('74268')or str(codecom1).startswith('74093') or str(codecom1).startswith('74217'):
        return('74010')
    elif str(codecom1).startswith('49199'):
        return('52400')
    elif str(codecom1).startswith('29052'):
        return('29003')
    elif str(codecom1).startswith('85060'):
        return('85194')
    elif str(codecom1).startswith('50602') or str(codecom1).startswith('50173') or str(codecom1).startswith('50203'):
        return('50129')
    elif str(codecom1).startswith('49285'):
        return('49301')
    elif str(codecom1).startswith('74181'):
        return('74112')
    elif str(codecom1).startswith('85107'):
        return('85146')
    elif str(codecom1).startswith('85166'):
        return('85194')
    elif str(codecom1).startswith('49375'):
        return('49023')
    elif str(codecom1).startswith('49065'):
        return('49080')
    elif str(codecom1).startswith('91182'):
        return('91228')
    elif str(codecom1).startswith('85069'):
        return('85008')
    elif str(codecom1).startswith('49276'):
        return('49244')
    elif str(codecom1).startswith('22038'):
        return('22206')
    elif str(codecom1).startswith('77491'):
        return('77316')
    elif str(codecom1).startswith('49273'):
        return('49301')
    else:
        return(str(codecom1))

In [590]:
liste_med_ban['codecommunecoordstructure3']=liste_med_ban['codecommunecoordstructure'].apply(codecom3)

In [591]:
liste_med_ban_metr=liste_med_ban[~(liste_med_ban['codecommunecoordstructure3'].str.startswith('97'))]

In [593]:
apl=pd.read_excel('data/APL_mg_2022.xlsx')

In [594]:
apl.drop(columns=["APL aux médecins généralistes (sans borne d'âge)","APL aux médecins généralistes de 62 ans et moins"],inplace=True)

In [595]:
apl.head()

,Code commune,Libellé de la commune,Population totale,APL aux médecins généralistes de 65 ans et moins,Population standardisée par la consommation de soins par tranche d'âge
0,01001,L'Abergement-Clémenciat,806.0,1.823,817.600
1,01002,L'Abergement-de-Varey,262.0,2.008,253.836
2,01004,Ambérieu-en-Bugey,14288.0,2.866,14028.085
3,01005,Ambérieux-en-Dombes,1782.0,3.922,1745.171
4,01006,Ambléon,113.0,0.799,120.879


In [596]:
apl=apl[~(apl['Code commune'].str.startswith('97'))]

In [597]:
apl['Code commune2'] = apl['Code commune'].apply(codecom3)

In [598]:
moyM=apl[apl['Code commune2']=='13055']['APL aux médecins généralistes de 65 ans et moins'].mean()
moyL=apl[apl['Code commune2']=='69123']['APL aux médecins généralistes de 65 ans et moins'].mean()
moyP=apl[apl['Code commune2']=='75056']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy74010=apl[apl['Code commune2']=='74010']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy52400=apl[apl['Code commune2']=='52400']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy29003=apl[apl['Code commune2']=='29003']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy85194=apl[apl['Code commune2']=='85194']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy50129=apl[apl['Code commune2']=='50129']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy49301=apl[apl['Code commune2']=='49301']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy74112=apl[apl['Code commune2']=='74112']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy85146=apl[apl['Code commune2']=='85146']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy85194=apl[apl['Code commune2']=='85194']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy49023=apl[apl['Code commune2']=='49023']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy49080=apl[apl['Code commune2']=='49080']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy91228=apl[apl['Code commune2']=='91228']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy85008=apl[apl['Code commune2']=='85008']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy49244=apl[apl['Code commune2']=='49244']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy22206=apl[apl['Code commune2']=='22206']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy77316=apl[apl['Code commune2']=='77316']['APL aux médecins généralistes de 65 ans et moins'].mean()
moy49301=apl[apl['Code commune2']=='49301']['APL aux médecins généralistes de 65 ans et moins'].mean()


In [599]:
def new_apl(apl,code):
    if code=='13055':
        return moyM
    elif code=='69123':
        return moyL
    elif code=='75056':
        return moyP
    elif code=='74010':
        return moy74010
    elif code=='52400':
        return moy52400
    elif code=='29003':
        return moy29003
    elif code=='85194':
        return moy85194
    elif code=='50129':
        return moy50129
    elif code=='49301':
        return moy49301
    elif code=='74112':
        return moy74112
    elif code=='85146':
        return moy85146
    elif code=='85194':
        return moy85194
    elif code=='49023':
        return moy49023
    elif code=='49080':
        return moy49080
    elif code=='91228':
        return moy91228
    elif code=='85008':
        return moy85008
    elif code=='49244':
        return moy49244
    elif code=='22206':
        return moy22206
    elif code=='77316':
        return moy77316
    elif code=='49301':
        return moy49301
    else:
        return apl

In [600]:
apl["APL"]=apl.apply(lambda x : new_apl(x['APL aux médecins généralistes de 65 ans et moins'],x['Code commune2']),axis=1)

In [601]:
apl_unique=apl.drop_duplicates(subset=['Code commune2'])
apl_unique.head() 

,Code commune,Libellé de la commune,Population totale,APL aux médecins généralistes de 65 ans et moins,Population standardisée par la consommation de soins par tranche d'âge,Code commune2,APL
0,01001,L'Abergement-Clémenciat,806.0,1.823,817.600,01001,1.823
1,01002,L'Abergement-de-Varey,262.0,2.008,253.836,01002,2.008
2,01004,Ambérieu-en-Bugey,14288.0,2.866,14028.085,01004,2.866
3,01005,Ambérieux-en-Dombes,1782.0,3.922,1745.171,01005,3.922
4,01006,Ambléon,113.0,0.799,120.879,01006,0.799


In [602]:
liste_med_ban_apl = liste_med_ban_metr.merge(apl_unique,how='inner', left_on='codecommunecoordstructure3', right_on='Code commune2')

In [604]:
#Compute the errors' ratio
(liste_med_ban_metr.shape[0]-liste_med_ban_apl.shape[0])/liste_med_ban_metr.shape[0]

0.002468998807112936

In [605]:
#To find where the errors are
liste_med_ban_apl_outer = liste_med_ban_metr.merge(apl_unique, how='outer', left_on='codecommunecoordstructure3', right_on='Code commune2', indicator=True)
#Filter lines without any match
liste_med_ban_no_match = liste_med_ban_apl_outer[liste_med_ban_apl_outer['_merge'] == 'left_only']
apl_no_match = liste_med_ban_apl_outer[liste_med_ban_apl_outer['_merge'] == 'right_only']

In [606]:
liste_med_ban_no_match['codecommunecoordstructure3'].value_counts()

(178,)

# Step 8: We select an instrumental variable 

In [607]:
fibre=pd.read_excel('data/fibre.xlsx')

/Users/julie/miniconda3/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/julie/miniconda3/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [608]:
fibre.head()

,Code Insee,Commune,Nombre de locaux,Fibre
0,25008,Aibre,214,0.981308
1,56194,Rieux,1892,0
2,70361,Montessaux,92,0.75
3,25262,Fuans,295,0.928814
4,89168,Fleys,167,0


In [609]:
fibre.drop(columns=['Nombre de locaux','Commune'],inplace=True)

In [610]:
#First attempt to merge
liste_med_ban_fibre_outer = liste_med_ban.merge(fibre, how='outer', left_on='codecommunecoordstructure3', right_on='Code Insee', indicator=True)
#Filter the lines without any match in the data frame
liste_med_ban_fibre_no_match = liste_med_ban_fibre_outer[liste_med_ban_fibre_outer['_merge'] == 'left_only']
fibre_no_match = liste_med_ban_fibre_outer[liste_med_ban_fibre_outer['_merge'] == 'right_only']

In [611]:
liste_med_ban_fibre_no_match.shape

(178, 77)

In [612]:
liste_med_ban_fibre = liste_med_ban_apl.merge(fibre, how='inner', left_on='codecommunecoordstructure3', right_on='Code Insee')

In [614]:
liste_med_ban_drop=liste_med_ban_fibre.drop(columns=['Code commune','Libellé de la commune','Code commune2','APL aux médecins généralistes de 65 ans et moins', 'Code Insee'])

# Step 9: We compute the percentage of doctors registered in Doctolib 

In [615]:
dico_share={}
for i in range(liste_med_ban_drop.shape[0]):
    codecom = liste_med_ban_drop['codecommunecoordstructure3'].iloc[i]
    if codecom not in dico_share.keys():
        share=1-(liste_med_ban_drop['RPPS'][liste_med_ban_drop['codecommunecoordstructure3']==codecom].isnull().sum()/liste_med_ban_drop[(liste_med_ban_drop['codecommunecoordstructure3']==codecom)].shape[0])
        dico_share[codecom]=share
        liste_med_ban_drop.loc[i,'share_Doctolib']=share
    else:
        liste_med_ban_drop.loc[i,'share_Doctolib']=dico_share[codecom]

In [617]:
df_a_merge=liste_med_ban_drop[['Population totale',"Population standardisée par la consommation de soins par tranche d'âge",'identifiantpp','APL','share_Doctolib','Fibre']]

In [618]:
df_a_merge['identifiantpp'].value_counts()

identifiantpp
10101216785    11
10100876696    11
10100945863     8
10100963874     8
10101490422     8
               ..
10002981131     1
10002981180     1
10002981701     1
10002982105     1
10110318770     1
Name: count, Length: 62643, dtype: int64

In [619]:
medecin_gen1=medecin_gen.merge(df_a_merge,how='inner',right_on='identifiantpp',left_on='RPPS')

In [620]:
medecin_gen1.drop(columns=['identifiantpp'],inplace=True)

In [621]:
medecin_gen1.shape

(46, 47)

# Step 10: Exportation 

In [622]:
liste_med.to_csv('liste_med.csv')
medecin_gen3.to_csv('medecin_gen.csv')
liste_med_ban.to_csv('liste_med_ban.csv')